In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from pydts.utils import get_expanded_df
from tqdm import tqdm
from time import time
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from lifelines.fitters.coxph_fitter import CoxPHFitter
from scipy.special import expit
from scipy.optimize import minimize
import sys
sys.path.append('../')
sys.path.append('../../')
from examples.generate_simulations_data import generate_quick_start_df

In [ ]:
n_cov = 5
patients_df = generate_quick_start_df(n_patients=10000, n_cov=n_cov, d_times=30, j_events=2, pid_col='pid', 
                                      seed=0)
covariates = [f'Z{i + 1}' for i in range(n_cov)]

train_df, test_df = train_test_split(patients_df, test_size=0.25)
events = sorted(train_df['J'].unique())
times = sorted(train_df['X'].unique())
train_df.head()  

# Lee et al. 2018

In [ ]:
expanded_train_df = get_expanded_df(train_df)
print(expanded_train_df.shape)
expanded_train_df

In [ ]:
expanded_train_df = pd.concat([expanded_train_df, 
                               pd.get_dummies(expanded_train_df['X'], prefix='alpha')], axis=1)
expanded_train_df

In [ ]:
alpha_list = [col for col in expanded_train_df.columns if col[:5] == 'alpha']
models = {}

In [ ]:
start = time()

for e in tqdm(events[1:], total=len(events)-1):
    model = LogisticRegression()
    X, y =  expanded_train_df[covariates + alpha_list].values, expanded_train_df[f'j_{e}'].values
    model.fit(X, y)
    models[e] = model

end = time()

In [ ]:
print(f'Total training time: {end - start:.2f} seconds')

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(14,6))
ax = axes[0]
ax.set_title(r'$\alpha_{jt}$', fontsize=26)
ax.plot(models[1].coef_[0, n_cov:], label='J=1')
ax.plot(models[2].coef_[0, n_cov:], color='g', label='J=2')
ax.set_xlabel(r'Time', fontsize=16)
ax.set_ylabel(r'$\alpha_{t}$', fontsize=20)
ax.legend(loc='upper center', fontsize=14)
ax2 = ax.twinx()
ax2.hist(expanded_train_df['X'], color='r', alpha=0.3, bins=times)
ax2.set_ylabel('N patients', fontsize=16, color='red')
ax2.tick_params(axis='y', colors='red')

ax = axes[1]
ax.set_title(r'$\beta_{j}$', fontsize=26)
ax.bar(np.arange(1, n_cov+1), models[1].coef_[0, :n_cov], label='J=1', width=0.5)
ax.bar(np.arange(1, n_cov+1), models[2].coef_[0, :n_cov], color='g', label='J=2', align='edge', width=0.5)
ax.legend(loc='lower center', fontsize=14)

fig.tight_layout()

In [ ]:
# test mse

# New approach

In [ ]:
beta_models = {}

for e in events[1:]:
    strata_df = expanded_train_df[covariates + [f'j_{e}', 'X']].copy()
    strata_df['X_copy'] = expanded_train_df['X']

    beta_j_cox = CoxPHFitter()
    beta_j_cox.fit(df=strata_df[covariates+ ['X', 'X_copy', f'j_{e}']], duration_col='X', event_col=f'j_{e}', 
                   strata='X_copy')
    beta_models[e] = beta_j_cox
    print(beta_j_cox.params_)

Going back to the unexpanded data: 

In [ ]:
train_df.head()

In [ ]:
y_t = len(train_df['X']) - train_df['X'].value_counts().sort_index().cumsum()
y_t.head()

In [ ]:
n_jt = train_df.groupby(['J', 'X']).size().to_frame().reset_index()
n_jt.columns = ['J', 'X' ,'n_jt']
n_jt.head()

In [ ]:
def alpha_jt(x, df, y_t, beta_j, n_jt, t, duration_col='X' ):
    partial_df = df[df[duration_col] >= t]
    expit_add = (partial_df[covariates]*beta_j).sum(axis=1)
    return ((1/y_t)*np.sum(expit(x+expit_add)) - (n_jt/y_t))**2

In [ ]:
x0 = 0

In [ ]:
alpha_df = pd.DataFrame()
for e in events[1:]:
    n_et = n_jt[n_jt['J'] == e]
    n_et['opt_res'] = n_et.apply(lambda row: minimize(alpha_jt, x0, 
        args=(train_df, y_t.loc[row['X']], beta_models[e].params_,  row['n_jt'],  row['X'])), axis=1)
    n_et['success'] = n_et['opt_res'].apply(lambda val: val.success)
    n_et['alpha_jt'] = n_et['opt_res'].apply(lambda val: val.x[0])
    alpha_df = alpha_df.append(n_et, ignore_index=True)
alpha_df